In [ ]:
from database import DB, Messages
db = DB("sqlite:///dentropydaemon.sqlite")
user_messages = db.session.query(Messages).filter(Messages.txt_body != None).filter(Messages.msg_type == "text")
list_of_string = []
for i in user_messages:
    list_of_string.append(i.txt_body)

In [ ]:
list_of_string[0]

In [ ]:
list_of_string[1]

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [ ]:
def lemmatize_stemming(text):
#https://www.nltk.org/howto/stem.html    
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
list_of_string[0]

In [ ]:
print(preprocess(list_of_string[0]))

In [ ]:
print(preprocess(list_of_string[1]))

In [ ]:
topics=[]
for msg in list_of_string[:]:
    print('msg=',msg)
    print('topics=',preprocess(msg))
    topics.append(preprocess(msg))

In [ ]:
topics

# make the BOW

https://en.wikipedia.org/wiki/Bag-of-words_model

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(topics)
corpus = [dictionary.doc2bow(text) for text in topics]

In [ ]:
corpus

In [ ]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [ ]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

# next up

* add to the STOPWORDS
* more cleaning, etc

In [ ]:
gensim.parsing.preprocessing.STOPWORDS